# 삼성 DS-KAIST AI Expert 프로그램
## Multitask Learning

실습 일시: 2020년 7월 22/29일 (수), 13:30 - 17:30

담당 조교: 탁지훈 (jihoontack@kaist.ac.kr), 모상우 (swmo@kaist.ac.kr)

# Section 0. Introduction

본 실습에서는 TensorFlow와 Keras 프레임워크를 이용하여, multi-task learning에 관한 실습을 진행할 것입니다. multi-task learning은 여러가지 테스크를 동시에 학습함으로써 공통된 유용한 feature를 효과적으로 배우는 방법으로, 어떤 효과가 있는지 알아볼 예정입니다.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

### Hyperparamters

아래는 이번 실습에서 사용할 hyperparameter들을 사전 정의해 둔 것입니다.

In [ ]:
batch_size = 128
steps_per_epoch = 300
num_epochs = 10

# Section 1. Load Dataset (CIFAR-10, CIFAR-100)

이번 실습에서 사용할 dataset은 CIFAR-10과 CIFAR-100 입니다. 각 dataset은 총 50000개의 training image를 가지고 있고, 10000개의 test image를 가지고 있습니다. 또한, CIFAR-10의 이미지는 10개의 class중 하나에 속하고, CIFAR-100의 이미지는 100개의 class중 하나에 속하게 됩니다.

In [ ]:
cifar10 = tf.keras.datasets.cifar10
cifar100 = tf.keras.datasets.cifar100

In [ ]:
(x_train_c10, y_train_c10), (x_test_c10, y_test_c10) = cifar10.load_data()
(x_train_c100, y_train_c100), (x_test_c100, y_test_c100) = cifar100.load_data()

In [ ]:
print('CIFAR-10 ------')
print('트레이닝 샘플 수: %d' % x_train_c10.shape[0])
print('이미지 크기: %d x %d x %d' % (x_train_c10.shape[1], x_train_c10.shape[2], x_train_c10.shape[3]))
print('트레이닝 데이터 크기: %d x %d x %d x %d' % (*x_train_c10.shape, ))
print('')
print('CIFAR-100 ------')
print('트레이닝 샘플 수: %d' % x_train_c100.shape[0])
print('이미지 크기: %d x %d x %d' % (x_train_c100.shape[1], x_train_c100.shape[2], x_train_c100.shape[3]))
print('트레이닝 데이터 크기: %d x %d x %d x %d' % (*x_train_c100.shape, ))

### Preprocessing Data
Image의 pixel 의 값은 (R, G, B) 형태로 표현되고, 각 channel (R/G/B) 마다 [0, 1, 2, ..., 255] 의 정수 값을 가집니다. 일반적으로 neural network를 학습할 때에는 normalized 되어있는 값을 input으로 사용하는 것이 좋습니다. 따라서, 각 image 값들을 [0, 1/255, ..., 1.0] 의 실수 값으로 치환해줍니다.
Image의 label 은 [0, 1, 2, ..., 9] 의 값을 가집니다. 이러한 label 정보는 일반적으로 one-hot encoding으로 바꿔주어 사용합니다. one-hot encoding을 하게 되면 label 0은 [1, 0, 0, ..., 0], label 1은 [0, 1, 0, ..., 0], 그리고 label 9는 [0, 0, 0, ..., 1]으로 바뀌게 됩니다.

In [ ]:
x_train_c10 = x_train_c10 / 255.0
x_test_c10 = x_test_c10 / 255.0

y_train_c10 = tf.keras.utils.to_categorical(y_train_c10, 10)
y_test_c10 = tf.keras.utils.to_categorical(y_test_c10, 10)

In [ ]:
x_train_c100 = x_train_c100 / 255.0
x_test_c100 = x_test_c100 / 255.0

y_train_c100 = tf.keras.utils.to_categorical(y_train_c100, 100)
y_test_c100 = tf.keras.utils.to_categorical(y_test_c100, 100)

### Data Generator

`ImageDataGenerator`는 Keras에 있는 API 중 하나로, data augmentation (좌우 반전 등) 을 손쉽게 할 수 있습니다. 가능한 data augmentation 방법들은 내용은 https://keras.io/preprocessing/image/ 에서 확인하실 수 있습니다. 이번 실습에서는 일반적으로 CIFAR dataset에서 사용하는 random 상하좌우 shift와 좌우 반전을 사용할 것입니다.

In [ ]:
datagen_c10 = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=5,
    height_shift_range=5,
    fill_mode='constant',
    horizontal_flip=True).flow(x_train_c10, y_train_c10, batch_size=batch_size)

In [ ]:
datagen_c100 = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=5,
    height_shift_range=5,
    fill_mode='constant',
    horizontal_flip=True).flow(x_train_c100, y_train_c100, batch_size=batch_size)

Test data를 위한 DataGenerator를 만들기 위해 아래의 함수를 선언하였습니다. Batch를 앞에서부터 100개씩 구성합니다.

In [ ]:
def test_datagen(x, y):
    assert x.shape[0] == 10000
    assert y.shape[0] == 10000
    while True:
        for i in range(0, 10000, 100):
            yield x[i:i+100], y[i:i+100]

In [ ]:
test_datagen_c10 = test_datagen(x_test_c10, y_test_c10)
test_datagen_c100 = test_datagen(x_test_c100, y_test_c100)

Multi-task learning을 위해서는, 2개의 data generator를 하나의 data generator로 합쳐줘야합니다. 아래의 함수를 통해 두개의 generator를 합칠 수 있습니다.

In [ ]:
def merge_datagen(datagen1, datagen2):
    while True:
        x1, y1 = next(datagen1)
        x2, y2 = next(datagen2)
        yield [x1, x2], [y1, y2]

In [ ]:
datagen = merge_datagen(datagen_c10, datagen_c100)
test_datagen = merge_datagen(test_datagen_c10, test_datagen_c100)

### Plotting Images

아래의 script를 이용하여 각 데이터셋에 어떤 이미지들이 들어있는지 확인할 수 있습니다.

In [ ]:
plt.figure(figsize=(6,6))
plt.suptitle('CIFAR-10 training images', fontsize=16)
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(x_train_c10[i])
    plt.axis('off')
    
plt.figure(figsize=(6,6))
plt.suptitle('CIFAR-100 training images', fontsize=16)
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(x_train_c100[i])
    plt.axis('off')

# Section 2. Multi-task Learning using Keras

이번 실습에서는 간단한 6-layer CNN을 사용할 것입니다. multi-task learning을 위해서, 2 종류의 inputs을 받고, 2 종류의 outputs을 출력하는 형태로 network를 design할 것입니다.

`build_shared_conv_block`은 convolutional layer와, batch normalization layer, 그리고 relu activation으로 구성된 하나의 convolutional block입니다.
두 개의 입력값 `inputs1`, `inputs2` 은 같은 layer를 공유해야하기 때문에, Keras API로 먼저 layer을 선언하고, 선언된 layer를 이용하여 출력값을 구해줍니다.

In [ ]:
def build_shared_conv_block(inputs1, inputs2, num_channels):
    # define layers
    conv = Conv2D(num_channels, (3, 3), padding='same')
    bn = BatchNormalization()
    relu = Activation('relu')
    
    # compute outputs
    outputs1 = relu(bn(conv(inputs1)))
    outputs2 = relu(bn(conv(inputs2)))
    return outputs1, outputs2

`build_network`는 multi-task를 다룰 수 있는 하나의 CNN을 만드는 역할을 합니다. 이 함수는 `inputs1`, `inputs2`, `num_classes1`, `num_classes2`, `base_width` 다섯 개의 인자를 받습니다.
- `inputs1`, `inputs2`: 각 task의 input을 의미합니다.
- `num_classes1`, `num_classes2`: 각 task의 output dimension을 의미합니다. CIFAR-10, CIFAR-100을 사용할 것이므로, 10 / 100 이 됩니다.
- `base_width`: network의 width를 나타내는 역할을 합니다. teacher model로 wide한 모델을 사용할 것이고, student 모델로는 thin한 모델을 사용할 예정입니다.

`build_network`는 총 2개의 tensor를 return합니다.
- `outputs1`, `outputs2`: network의 최종 출력값으로, 각 task에 대한 output을 의미합니다.

## Q1. Build Separated Dense Layers

아래의 코드에서 `x1, x2`는 layer를 공유하는 형태로 계산된 feature입니다. 여기서 `Dense` layer를 이용하되, 하나의 task 마다 하나의 `Dense` layer가 생기도록 구현해보세요.
각 `Dense` layer의 output dimension은 `num_classes1`과 `num_classes2`가 되어야 합니다. 비슷하게, Softmax activation도 구현하세요.

Transfer learning 실습 코드를 참고하면 좋습니다.

In [ ]:
def build_network(inputs1, inputs2, num_classes1=10, num_classes2=100, base_width=16):
    x1, x2 = build_shared_conv_block(inputs1, inputs2, base_width)
    x1, x2 = build_shared_conv_block(x1, x2, base_width)
    
    pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')
    x1, x2 = pool1(x1), pool1(x2)
    x1, x2 = build_shared_conv_block(x1, x2, base_width*2)
    x1, x2 = build_shared_conv_block(x1, x2, base_width*2)

    pool2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')
    x1, x2 = pool2(x1), pool2(x2)
    x1, x2 = build_shared_conv_block(x1, x2, base_width*4)
    x1, x2 = build_shared_conv_block(x1, x2, base_width*4)

    pool3 = GlobalAveragePooling2D()
    x1, x2 = pool3(x1), pool3(x2)
    
    # Q1. Separated Dense layers
    logits1 = ?
    logits2 = ?
    
    outputs1 = ?
    outputs2 = ?
    
    return outputs1, outputs2

In [ ]:
inputs1 = Input(shape=(32, 32, 3), name='inputs1')
inputs2 = Input(shape=(32, 32, 3), name='inputs2')

## Define Model

먼저, `base_width=16`인 network를 정의합니다. 이 모델은 inputs이 2개, outputs이 2개인 모델입니다.

In [ ]:
outputs1, outputs2 = build_network(inputs1, inputs2, base_width=8)
model = tf.keras.models.Model(inputs=[inputs1, inputs2], outputs=[outputs1, outputs2], name='model')
model.summary()

아래 함수 `plot_history`는 training curve를 시각화하는 함수입니다. 학습 후 성능이 어떻게 변화하는지 관측해볼 것입니다.

In [ ]:
def plot_history(history):
    epochs = len(history.history['loss'])
    plt.figure(figsize=(20, 3))

    plt.subplot(1, 4, 1)
    plt.title('loss')
    plt.plot(range(epochs), history.history['outputs1_loss'], 'r')
    plt.plot(range(epochs), history.history['val_outputs1_loss'], 'b')
    plt.legend(['train', 'val'])

    plt.subplot(1, 4, 2)
    plt.title('accuracy')
    plt.plot(range(epochs), history.history['outputs1_acc'], 'r--')
    plt.plot(range(epochs), history.history['val_outputs1_acc'], 'b--')    
    plt.legend(['train', 'val'])

    plt.subplot(1, 4, 3)
    plt.title('loss')
    plt.plot(range(epochs), history.history['outputs2_loss'], 'r')
    plt.plot(range(epochs), history.history['val_outputs2_loss'], 'b')
    plt.legend(['train', 'val'])

    plt.subplot(1, 4, 4)
    plt.title('accuracy')
    plt.plot(range(epochs), history.history['outputs2_acc'], 'r--')
    plt.plot(range(epochs), history.history['val_outputs2_acc'], 'b--')    
    plt.legend(['train', 'val'])

    print('CIFAR10  최대 도달 성능: %.2f%%' % (max(history.history['val_outputs1_acc'])*100,))
    print('CIFAR100 최대 도달 성능: %.2f%%' % (max(history.history['val_outputs2_acc'])*100,))

## Train Model on CIFAR-10 & CIFAR-100

두 가지 데이터를 동시에 학습해야 하므로, 다음과 같이 compile해줍니다.

In [ ]:
model.compile(optimizer='adam',
              loss=['categorical_crossentropy', 'categorical_crossentropy'], # output이 2개이기 때문에 각각에 대해 loss를 지정해주어야 합니다.
              metrics=['accuracy', 'accuracy']) # output이 2개이기 때문에 각각에 대해 metric을 지정해주어야 합니다.

그리고 Model의 `fit_generator`를 사용하여 네트워크를 학습해줍니다. 여기서 `generator`는 위에서 선언한 data augmentation이 된 dataset으로 생각할 수 있습니다. `steps_per_epoch`은 한 epoch마다 iteration 수를, `epochs`는 총 epoch을 나타냅니다.

In [ ]:
history = model.fit_generator(datagen,
                              steps_per_epoch=steps_per_epoch,
                              epochs=num_epochs,
                              validation_data=test_datagen,
                              validation_steps=100, verbose=1)

위에서 선언한 `plot_history` 함수를 이용하여 학습이 어떻게 되었는지 확인해봅니다.

In [ ]:
plot_history(history)

위에서 정의한 모델은 앞의 transfer learning에서 사용한 student model과 같은 크기의 모델입니다. cifar10에서와 cifar100에서의 모델 성능을 확인해보세요. 하나의 모델만 사용했음에도, 두가지 테스크에서 어느정도 성능을 보여줍니다.

## Q2. BN layers를 분리하기

아래의 두 함수를 수정하여, BN layer가 각 task별로 따로 사용될 수 있도록 분리해보세요.

In [ ]:
def build_shared_conv_block(inputs1, inputs2, num_channels):
    # Q2. 이 함수를 수정하세요!
    
    # define layers
    conv = Conv2D(num_channels, (3, 3), padding='same')
    bn = BatchNormalization()
    relu = Activation('relu')
    
    # compute outputs
    outputs1 = relu(bn(conv(inputs1)))
    outputs2 = relu(bn(conv(inputs2)))
    return outputs1, outputs2

이전 실습과 위의 코드를 바탕으로, `model`을 다시 build(`build_network` 이용)하고, compile한 뒤, 학습을 다시 해보세요

In [ ]:
outputs1, outputs2 = build_network(inputs1, inputs2, base_width=8)
model = tf.keras.models.Model(inputs=[inputs1, inputs2], outputs=[outputs1, outputs2], name='model')
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=['categorical_crossentropy', 'categorical_crossentropy'], # output이 2개이기 때문에 각각에 대해 loss를 지정해주어야 합니다.
              metrics=['accuracy', 'accuracy']) # output이 2개이기 때문에 각각에 대해 metric을 지정해주어야 합니다.

In [ ]:
history = model.fit_generator(datagen,
                              steps_per_epoch=steps_per_epoch,
                              epochs=num_epochs,
                              validation_data=test_datagen,
                              validation_steps=100, verbose=1)

In [ ]:
plot_history(history)